In [1]:
#Author: ML Tlachac, WPI
#For DepreST-CAT, 2021

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectKBest, chi2
from sklearn import preprocessing
from scipy import stats
import collections
import operator
import argparse
import random
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn import metrics
from statistics import mean 
from sklearn.naive_bayes import GaussianNB
from sklearn.utils import resample
from sklearn import preprocessing
from sklearn import utils
from sklearn.datasets import load_digits
from sklearn import svm
import xgboost as xgb
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import random
from sklearn.decomposition import PCA, KernelPCA, NMF

C:\Users\mltla\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\mltla\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\mltla\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


In [42]:
labels = ["phq9", "gad7"]
modelTypelist = ["SVC", "kNN", "RF", "LR", "XG"]
splits = [5,6,7,8,9,10]

for split in splits: 
    
    for week in [2,4,8,16]:

        data = pd.read_csv("featureSet" + str(week) + "weeksDepreST-CAT.csv")
        print(data.shape)
        
        n = int((data.shape[1]-3)/2)
        
        cids = range(n+1, data.shape[1]-2)
        tids = range(1, n+1)
        
        cnames = ['id'] + list(data.columns[cids]) + ['phq9','gad7']
        tnames = ['id'] + list(data.columns[tids]) + ['phq9','gad7']
                
        calls = data[cnames]
        texts = data[tnames]
        
        data = texts
        print(data.shape)

        #binary labels
        d10 = []
        g10 = []
        for i in range(0, data.shape[0]):
            if int(data.phq9[i]) >= split:
                d10.append(1)
            else:
                d10.append(0)
            if int(data.gad7[i]) >= split:
                g10.append(1)
            else:
                g10.append(0)
        data["phq9"] = d10
        data["gad7"] = g10
        
        for label in labels:

            #create lists to populate
            flist = [] 
            mlist = []
            llist = []
            featureList = []
            wlist = []
            slist = []
            f1List = []
            accuracyList = []
            truePosList = []
            trueNegList = []
            falsePosList = []
            falseNegList = []
            predictions = []
            rseed = []

            for r in range(0, 100):

                #train/test split    
                df_train, df_test = train_test_split(data, test_size=0.3, stratify=data[["phq9", "gad7"]], random_state = r)
                trainids = list(df_train["id"])
                testids = list(df_test["id"])
                print(data.shape)
                testdata = data[data['id'].isin(testids)]
                print(testdata.shape)
                traindata = data[data['id'].isin(trainids)]
                print(traindata.shape)

                #limit to features
                testContent = testdata[testdata.columns[1:-2]]
                print(testContent.shape)
                trainContent = traindata[traindata.columns[1:-2]]
                print(trainContent.shape)

                #NEED TO SCALE BEFORE FEATURE SELECTION/REDUCATION
                min_max_scaler = preprocessing.MinMaxScaler()  
                np_scaled = min_max_scaler.fit_transform(trainContent)
                featureSubset = pd.DataFrame(np_scaled)
                np_scaled2 =  min_max_scaler.transform(testContent)
                testSubset = pd.DataFrame(np_scaled2)
                print(featureSubset.shape)
                print(testSubset.shape)

                target = list(traindata[label])

                featureDF = []
                testDFs = []

                nFeatureList = list(np.arange(1,5,1))
                for numberOfFeatures in nFeatureList:
                    pca = PCA(n_components=numberOfFeatures)
                    pca = pca.fit(featureSubset)
                    X_pca = pca.transform(featureSubset)
                    pcaDF = pd.DataFrame(X_pca)
                    pcaDF = pcaDF.assign(target = target)
                    featureDF.append(pcaDF)
                    testSubset2 = pca.transform(testSubset)
                    testDFs.append(pd.DataFrame(testSubset2))


                for f in range(0, len(featureDF)):
                    print(f)

                    train_phq9 = featureDF[f]
                    X_test = testDFs[f]

                    # upsampling 
                    #Count 1s and 0s
                    ones = len(train_phq9.loc[train_phq9['target'] == 1])
                    zeros = len(train_phq9.loc[train_phq9['target'] == 0])
                    if ones >= zeros:
                        majority = 1
                        minority = 0
                    else:
                        majority = 0
                        minority = 1


                    # Upsample TrainingSet 
                    train_majority = train_phq9[train_phq9.target==majority]
                    train_minority = train_phq9[train_phq9.target==minority]

                    #print("train_majority ="  + str(len(train_majority)))
                    #print("train_minority ="  + str(len(train_minority)))

                    # Upsample minority class
                    train_minority_upsampled = resample(train_minority, 
                                                     replace=True,     # sample with replacement
                                                     n_samples=len(train_majority),    # to match majority class
                                                     random_state=42) # reproducible results

                    # Combine majority class with upsampled minority class
                    train_phq9 = pd.concat([train_majority, train_minority_upsampled])

                    #seperate features and target
                    y_train = train_phq9["target"]
                    X_train = train_phq9.drop(columns = "target")

                    for modelType in modelTypelist:

                        #add data to lists
                        llist.append(label)
                        featureList.append(f +1)
                        flist.append("PCA")
                        mlist.append(modelType)

                        #chose model type
                        if modelType == "SVC":
                            clf = svm.SVC(random_state=r)
                        elif modelType == "RF":
                            clf = RandomForestClassifier(random_state=r)
                        elif modelType == "kNN":
                            clf = KNeighborsClassifier()
                        elif modelType == "LR":
                            clf = LogisticRegression(random_state=r)
                        elif modelType =="XG":
                            clf = xgb.XGBClassifier(random_state=r)

                        #train model and make predictions
                        clf.fit(X_train, y_train)
                        y_pred = clf.predict(X_test)

                        #evaluate model
                        conf_mat = confusion_matrix(list(testdata[label]), y_pred)
                        TN = conf_mat[0][0]
                        TP = conf_mat[1][1]
                        FP = conf_mat[0][1]
                        FN = conf_mat[1][0]
                        precision = TP/(TP+FP)
                        sensitivity = TP/(TP+FN)
                        f1 = (2*precision*sensitivity)/(precision + sensitivity)
                        accuracy = (TP+TN)/(TN+TP+FP+FN)

                        #populate lists with results
                        f1List.append(f1)
                        accuracyList.append(accuracy)
                        truePosList.append(TP)
                        trueNegList.append(TN)
                        falsePosList.append(FP)
                        falseNegList.append(FN)
                        predictions.append(y_pred)
                        rseed.append(r)
                        slist.append(split)
                        wlist.append(week)

            resultsDF = pd.DataFrame()
            resultsDF["week"] = wlist
            resultsDF["label"] = llist
            resultsDF["split"] = slist
            resultsDF["Engineering"] = flist
            resultsDF["model"] = mlist
            resultsDF["nFeatures"] = featureList
            resultsDF["F1"] = f1List
            resultsDF["Accuracy"] = accuracyList
            resultsDF["truePos"] = truePosList
            resultsDF["trueNeg"] = trueNegList
            resultsDF["falsePos"] = falsePosList
            resultsDF["falseNeg"] = falseNegList
            resultsDF["predictions"] = predictions
            resultsDF["randomSeed"] = rseed

            resultsDF.to_csv("resultsText/resultsCAT" + str(week) + "week" + label + "split" + str(split) + ".csv")

(369, 571)
(369, 287)
(369, 287)
(111, 287)
(258, 287)
(111, 284)
(258, 284)
(258, 284)
(111, 284)
0
1
2
3
(369, 287)
(111, 287)
(258, 287)
(111, 284)
(258, 284)
(258, 284)
(111, 284)
0
1
2
3
(369, 287)
(111, 287)
(258, 287)
(111, 284)
(258, 284)
(258, 284)
(111, 284)
0
1
2
3
(369, 287)
(111, 287)
(258, 287)
(111, 284)
(258, 284)
(258, 284)
(111, 284)
0
1
2
3
(369, 287)
(111, 287)
(258, 287)
(111, 284)
(258, 284)
(258, 284)
(111, 284)
0
1
2
3
(369, 287)
(111, 287)
(258, 287)
(111, 284)
(258, 284)
(258, 284)
(111, 284)
0
1
2
3
(369, 287)
(111, 287)
(258, 287)
(111, 284)
(258, 284)
(258, 284)
(111, 284)
0
1
2
3
(369, 287)
(111, 287)
(258, 287)
(111, 284)
(258, 284)
(258, 284)
(111, 284)
0
1
2
3
(369, 287)
(111, 287)
(258, 287)
(111, 284)
(258, 284)
(258, 284)
(111, 284)
0
1
2
3
(369, 287)
(111, 287)
(258, 287)
(111, 284)
(258, 284)
(258, 284)
(111, 284)
0
1
2
3
(369, 287)
(111, 287)
(258, 287)
(111, 284)
(258, 284)
(258, 284)
(111, 284)
0
1
2
3
(369, 287)
(111, 287)
(258, 287)
(111, 284)

0
1
2
3
(369, 287)
(111, 287)
(258, 287)
(111, 284)
(258, 284)
(258, 284)
(111, 284)
0
1
2
3
(369, 287)
(111, 287)
(258, 287)
(111, 284)
(258, 284)
(258, 284)
(111, 284)
0
1
2
3
(369, 287)
(111, 287)
(258, 287)
(111, 284)
(258, 284)
(258, 284)
(111, 284)
0
1
2
3
(369, 287)
(111, 287)
(258, 287)
(111, 284)
(258, 284)
(258, 284)
(111, 284)
0
1
2
3
(369, 287)
(111, 287)
(258, 287)
(111, 284)
(258, 284)
(258, 284)
(111, 284)
0
1
2
3
(369, 287)
(111, 287)
(258, 287)
(111, 284)
(258, 284)
(258, 284)
(111, 284)
0
1
2
3
(369, 287)
(111, 287)
(258, 287)
(111, 284)
(258, 284)
(258, 284)
(111, 284)
0
1
2
3
(369, 287)
(111, 287)
(258, 287)
(111, 284)
(258, 284)
(258, 284)
(111, 284)
0
1
2
3
(369, 287)
(111, 287)
(258, 287)
(111, 284)
(258, 284)
(258, 284)
(111, 284)
0
1
2
3
(369, 287)
(111, 287)
(258, 287)
(111, 284)
(258, 284)
(258, 284)
(111, 284)
0
1
2
3
(369, 287)
(111, 287)
(258, 287)
(111, 284)
(258, 284)
(258, 284)
(111, 284)
0
1
2
3
(369, 287)
(111, 287)
(258, 287)
(111, 284)
(258, 284)
(2

0
1
2
3
(369, 287)
(111, 287)
(258, 287)
(111, 284)
(258, 284)
(258, 284)
(111, 284)
0
1
2
3
(369, 287)
(111, 287)
(258, 287)
(111, 284)
(258, 284)
(258, 284)
(111, 284)
0
1
2
3
(369, 287)
(111, 287)
(258, 287)
(111, 284)
(258, 284)
(258, 284)
(111, 284)
0
1
2
3
(369, 287)
(111, 287)
(258, 287)
(111, 284)
(258, 284)
(258, 284)
(111, 284)
0
1
2
3
(369, 287)
(111, 287)
(258, 287)
(111, 284)
(258, 284)
(258, 284)
(111, 284)
0
1
2
3
(369, 287)
(111, 287)
(258, 287)
(111, 284)
(258, 284)
(258, 284)
(111, 284)
0
1
2
3
(369, 599)
(369, 301)
(369, 301)
(111, 301)
(258, 301)
(111, 298)
(258, 298)
(258, 298)
(111, 298)
0
1
2
3
(369, 301)
(111, 301)
(258, 301)
(111, 298)
(258, 298)
(258, 298)
(111, 298)
0
1
2
3
(369, 301)
(111, 301)
(258, 301)
(111, 298)
(258, 298)
(258, 298)
(111, 298)
0
1
2
3
(369, 301)
(111, 301)
(258, 301)
(111, 298)
(258, 298)
(258, 298)
(111, 298)
0
1
2
3
(369, 301)
(111, 301)
(258, 301)
(111, 298)
(258, 298)
(258, 298)
(111, 298)
0
1
2
3
(369, 301)
(111, 301)
(258, 301)
(1

0
1
2
3
(369, 301)
(111, 301)
(258, 301)
(111, 298)
(258, 298)
(258, 298)
(111, 298)
0
1
2
3
(369, 301)
(111, 301)
(258, 301)
(111, 298)
(258, 298)
(258, 298)
(111, 298)
0
1
2
3
(369, 301)
(111, 301)
(258, 301)
(111, 298)
(258, 298)
(258, 298)
(111, 298)
0
1
2
3
(369, 301)
(111, 301)
(258, 301)
(111, 298)
(258, 298)
(258, 298)
(111, 298)
0
1
2
3
(369, 301)
(111, 301)
(258, 301)
(111, 298)
(258, 298)
(258, 298)
(111, 298)
0
1
2
3
(369, 301)
(111, 301)
(258, 301)
(111, 298)
(258, 298)
(258, 298)
(111, 298)
0
1
2
3
(369, 301)
(111, 301)
(258, 301)
(111, 298)
(258, 298)
(258, 298)
(111, 298)
0
1
2
3
(369, 301)
(111, 301)
(258, 301)
(111, 298)
(258, 298)
(258, 298)
(111, 298)
0
1
2
3
(369, 301)
(111, 301)
(258, 301)
(111, 298)
(258, 298)
(258, 298)
(111, 298)
0
1
2
3
(369, 301)
(111, 301)
(258, 301)
(111, 298)
(258, 298)
(258, 298)
(111, 298)
0
1
2
3
(369, 301)
(111, 301)
(258, 301)
(111, 298)
(258, 298)
(258, 298)
(111, 298)
0
1
2
3
(369, 301)
(111, 301)
(258, 301)
(111, 298)
(258, 298)
(2

0
1
2
3
(369, 301)
(111, 301)
(258, 301)
(111, 298)
(258, 298)
(258, 298)
(111, 298)
0
1
2
3
(369, 301)
(111, 301)
(258, 301)
(111, 298)
(258, 298)
(258, 298)
(111, 298)
0
1
2
3
(369, 301)
(111, 301)
(258, 301)
(111, 298)
(258, 298)
(258, 298)
(111, 298)
0
1
2
3
(369, 301)
(111, 301)
(258, 301)
(111, 298)
(258, 298)
(258, 298)
(111, 298)
0
1
2
3
(369, 301)
(111, 301)
(258, 301)
(111, 298)
(258, 298)
(258, 298)
(111, 298)
0
1
2
3
(369, 301)
(111, 301)
(258, 301)
(111, 298)
(258, 298)
(258, 298)
(111, 298)
0
1
2
3
(369, 301)
(111, 301)
(258, 301)
(111, 298)
(258, 298)
(258, 298)
(111, 298)
0
1
2
3
(369, 301)
(111, 301)
(258, 301)
(111, 298)
(258, 298)
(258, 298)
(111, 298)
0
1
2
3
(369, 301)
(111, 301)
(258, 301)
(111, 298)
(258, 298)
(258, 298)
(111, 298)
0
1
2
3
(369, 301)
(111, 301)
(258, 301)
(111, 298)
(258, 298)
(258, 298)
(111, 298)
0
1
2
3
(369, 301)
(111, 301)
(258, 301)
(111, 298)
(258, 298)
(258, 298)
(111, 298)
0
1
2
3
(369, 301)
(111, 301)
(258, 301)
(111, 298)
(258, 298)
(2

...


2
3
(369, 329)
(111, 329)
(258, 329)
(111, 326)
(258, 326)
(258, 326)
(111, 326)
0
1
2
3
(369, 329)
(111, 329)
(258, 329)
(111, 326)
(258, 326)
(258, 326)
(111, 326)
0
1
2
3
(369, 329)
(111, 329)
(258, 329)
(111, 326)
(258, 326)
(258, 326)
(111, 326)
0
1
2
3
(369, 329)
(111, 329)
(258, 329)
(111, 326)
(258, 326)
(258, 326)
(111, 326)
0
1
2
3
(369, 329)
(111, 329)
(258, 329)
(111, 326)
(258, 326)
(258, 326)
(111, 326)
0
1
2
3
(369, 329)
(111, 329)
(258, 329)
(111, 326)
(258, 326)
(258, 326)
(111, 326)
0
1
2
3
(369, 329)
(111, 329)
(258, 329)
(111, 326)
(258, 326)
(258, 326)
(111, 326)
0
1
2
3
(369, 329)
(111, 329)
(258, 329)
(111, 326)
(258, 326)
(258, 326)
(111, 326)
0
1
2
3
(369, 329)
(111, 329)
(258, 329)
(111, 326)
(258, 326)
(258, 326)
(111, 326)
0
1
2
3
(369, 329)
(111, 329)
(258, 329)
(111, 326)
(258, 326)
(258, 326)
(111, 326)
0
1
2
3
(369, 329)
(111, 329)
(258, 329)
(111, 326)
(258, 326)
(258, 326)
(111, 326)
0
1
2
3
(369, 329)
(111, 329)
(258, 329)
(111, 326)
(258, 326)
(258,

0
1
2
3
(369, 329)
(111, 329)
(258, 329)
(111, 326)
(258, 326)
(258, 326)
(111, 326)
0
1
2
3
(369, 329)
(111, 329)
(258, 329)
(111, 326)
(258, 326)
(258, 326)
(111, 326)
0
1
2
3
(369, 329)
(111, 329)
(258, 329)
(111, 326)
(258, 326)
(258, 326)
(111, 326)
0
1
2
3
(369, 329)
(111, 329)
(258, 329)
(111, 326)
(258, 326)
(258, 326)
(111, 326)
0
1
2
3
(369, 329)
(111, 329)
(258, 329)
(111, 326)
(258, 326)
(258, 326)
(111, 326)
0
1
2
3
(369, 329)
(111, 329)
(258, 329)
(111, 326)
(258, 326)
(258, 326)
(111, 326)
0
1
2
3
(369, 329)
(111, 329)
(258, 329)
(111, 326)
(258, 326)
(258, 326)
(111, 326)
0
1
2
3
(369, 329)
(111, 329)
(258, 329)
(111, 326)
(258, 326)
(258, 326)
(111, 326)
0
1
2
3
(369, 329)
(111, 329)
(258, 329)
(111, 326)
(258, 326)
(258, 326)
(111, 326)
0
1
2
3
(369, 329)
(111, 329)
(258, 329)
(111, 326)
(258, 326)
(258, 326)
(111, 326)
0
1
2
3
(369, 329)
(111, 329)
(258, 329)
(111, 326)
(258, 326)
(258, 326)
(111, 326)
0
1
2
3
(369, 329)
(111, 329)
(258, 329)
(111, 326)
(258, 326)
(2

0
1
2
3
(369, 385)
(111, 385)
(258, 385)
(111, 382)
(258, 382)
(258, 382)
(111, 382)
0
1
2
3
(369, 385)
(111, 385)
(258, 385)
(111, 382)
(258, 382)
(258, 382)
(111, 382)
0
1
2
3
(369, 385)
(111, 385)
(258, 385)
(111, 382)
(258, 382)
(258, 382)
(111, 382)
0
1
2
3
(369, 385)
(111, 385)
(258, 385)
(111, 382)
(258, 382)
(258, 382)
(111, 382)
0
1
2
3
(369, 385)
(111, 385)
(258, 385)
(111, 382)
(258, 382)
(258, 382)
(111, 382)
0
1
2
3
(369, 385)
(111, 385)
(258, 385)
(111, 382)
(258, 382)
(258, 382)
(111, 382)
0
1
2
3
(369, 385)
(111, 385)
(258, 385)
(111, 382)
(258, 382)
(258, 382)
(111, 382)
0
1
2
3
(369, 385)
(111, 385)
(258, 385)
(111, 382)
(258, 382)
(258, 382)
(111, 382)
0
1
2
3
(369, 385)
(111, 385)
(258, 385)
(111, 382)
(258, 382)
(258, 382)
(111, 382)
0
1
2
3
(369, 385)
(111, 385)
(258, 385)
(111, 382)
(258, 382)
(258, 382)
(111, 382)
0
1
2
3
(369, 385)
(111, 385)
(258, 385)
(111, 382)
(258, 382)
(258, 382)
(111, 382)
0
1
2
3
(369, 385)
(111, 385)
(258, 385)
(111, 382)
(258, 382)
(2

0
1
2
3
(369, 385)
(111, 385)
(258, 385)
(111, 382)
(258, 382)
(258, 382)
(111, 382)
0
1
2
3
(369, 385)
(111, 385)
(258, 385)
(111, 382)
(258, 382)
(258, 382)
(111, 382)
0
1
2
3
(369, 385)
(111, 385)
(258, 385)
(111, 382)
(258, 382)
(258, 382)
(111, 382)
0
1
2
3
(369, 385)
(111, 385)
(258, 385)
(111, 382)
(258, 382)
(258, 382)
(111, 382)
0
1
2
3
(369, 385)
(111, 385)
(258, 385)
(111, 382)
(258, 382)
(258, 382)
(111, 382)
0
1
2
3
(369, 385)
(111, 385)
(258, 385)
(111, 382)
(258, 382)
(258, 382)
(111, 382)
0
1
2
3
(369, 385)
(111, 385)
(258, 385)
(111, 382)
(258, 382)
(258, 382)
(111, 382)
0
1
2
3
(369, 385)
(111, 385)
(258, 385)
(111, 382)
(258, 382)
(258, 382)
(111, 382)
0
1
2
3
(369, 385)
(111, 385)
(258, 385)
(111, 382)
(258, 382)
(258, 382)
(111, 382)
0
1
2
3
(369, 385)
(111, 385)
(258, 385)
(111, 382)
(258, 382)
(258, 382)
(111, 382)
0
1
2
3
(369, 385)
(111, 385)
(258, 385)
(111, 382)
(258, 382)
(258, 382)
(111, 382)
0
1
2
3
(369, 385)
(111, 385)
(258, 385)
(111, 382)
(258, 382)
(2

0
1
2
3
(369, 385)
(111, 385)
(258, 385)
(111, 382)
(258, 382)
(258, 382)
(111, 382)
0
1
2
3
(369, 385)
(111, 385)
(258, 385)
(111, 382)
(258, 382)
(258, 382)
(111, 382)
0
1
2
3


In [88]:
score = "F1"
for label in ["phq9", "gad7"]:
    print("______________" + label + "_______________")
    for week in ['2']:
        print("WEEK:" + week)
        df = pd.read_csv("results/resultsCAT" + str(week) + "week" + label + "split5.csv")
        for model in ["kNN"]:
            d = df[df.model == model]
            d1 = d[d.nFeatures == 1].fillna(0)
            print(model + " " +  str(score) + ": $" + str(round(sum(d1[score])/100,2)) + "\pm" + str(round(np.std(d1[score]),2)) + "$")

______________phq9_______________
WEEK:2
kNN F1: $0.68\pm0.04$
______________gad7_______________
WEEK:2
kNN F1: $0.63\pm0.05$
